Script d'analyse d'Alexandre Cléroux. Utilisé comme référence pour générer mes propres scripts.

## Modules

In [ ]:
import os
from tqdm import tqdm

import csv
import numpy as np
from scipy.signal import find_peaks, correlate, butter, sosfilt, freqs, filtfilt
from scipy.ndimage import gaussian_filter, median_filter, shift, uniform_filter
from scipy.stats import sem
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from skimage.registration import phase_cross_correlation

import seaborn as sns
import seaborn_image as isns
import cmcrameri.cm as cmc
from matplotlib import pyplot as plt
from matplotlib import colormaps as clm

from PIL import Image
import tifffile as tff

cmap = 'cmc.batlow'
# sns.set_palette(cmap)

## General functions for all modalities

In [ ]:
def identify_files(path, keywords):
    items = os.listdir(path)
    files = []
    for item in items:
        if all(keyword in item for keyword in keywords):
            files.append(item)
    files = [os.path.join(path, f) for f in files]
    files.sort(key=lambda x: os.path.getmtime(x))
    return files


def resample_pixel_value(data, bits):
    plage = 2**bits - 1
    return (plage * (data - np.min(data))/(np.max(data - np.min(data))))


def save_as_tiff(frames, prefixe, save_path):
    """_summary_

    Args:
        frames (array): 3D array of one type of data, ex HbO, HbR, or HbT
        prefixe (str): type of data
        save_path (str): folder to save data
    """
    for idx, frame in tqdm(enumerate(frames)):
        im = Image.fromarray(frame, mode='I;16')
        im.save(save_path + "\\{}.tiff".format(prefixe + str(idx)), "TIFF")


def create_list_trials(frames_timestamps:list, event_times:list, FPS:int=10): 
    """Creates a list of indices for frames of a same air puff trial, separated for each airpuff, with 3 seconds before, and 10 seconds after the airpuff.

    Args:
        frames_timestamps (list): array containing the timestamps of all the frames for one channel
        event_times (list): array containing the time stamps for the air puff delivery
        FPS (int): integer of the framerate or frequency of acquisition for one channel. Defaults to 10.

    Returns:
        list: a list containing all the indices of the frames, sorted by air puff
    """

    AP_idx = []
    for ti in event_times:
        AP_idx.append(np.argmin(np.absolute(frames_timestamps-ti)))

    AP_idx = AP_idx[:-1]
    Nf_bef = 3*FPS
    Nf_aft = 10*FPS

    sorted_frames_idx = []
    for idx in AP_idx:
        trial_idx = [idx-Nf_bef, idx+Nf_aft]
        sorted_frames_idx.append(trial_idx)

    return sorted_frames_idx



def create_npy_stack(folder_path:str, save_path:str,  wl:int, saving=False, cutAroundEvent=None):
    """creates a 3D npy stack of raw tiff images

    Args:
        folder_path (str): folder containing tiff frames
        save_path (str): folder to save npy stack
        wl (int): wavelength for saved file name
    """

    if cutAroundEvent is not None:
        files = cutAroundEvent
    
    else:
        files = identify_files(folder_path, "tif")
    
    # files=files[:250]
    for idx, file in tqdm(enumerate(files)):
        frame = tff.TiffFile(file).asarray()
        if idx == 0:
            num_frames = len(files)
            frame_shape = frame.shape
            stack_shape = (num_frames, frame_shape[0], frame_shape[1])
            _3d_stack = np.zeros(stack_shape, dtype=np.uint16)
        _3d_stack[idx,:,:] = frame

    if saving:
        np.save(save_path+"\\{}_rawStack.npy".format(wl), _3d_stack)
    return _3d_stack


def motion_correction(still_frame, frames):
    """Applies motion correction based on a phase cross correlation

    Args:
        frames (_type_): 3D array of frames before correction

    Returns:
        _type_: 3D array of frames after correction
    """
    fixed_frame = still_frame
    motion_corrected = np.zeros((frames.shape), dtype=np.uint16)
    for idx, frame in tqdm(enumerate(frames)):
        if idx == 0:
            motion_corrected[0,:,:] = frame
            continue
        shifted, error, diffphase = phase_cross_correlation(fixed_frame, frame, upsample_factor=10)
        corrected_image = shift(frame, shift=(shifted[0], shifted[1]), mode='reflect')
        motion_corrected[idx,:,:] = corrected_image
    
    return motion_corrected


def bin_pixels(frames, bin_size=2):
    """Bins pixels with bin size

    Args:
        frames (array): 3D array of frames. 
        bin_size (int, optional): size of pixel bins. Defaults to 2.

    Returns:
        array: 3D array, stack of binned data
    """
    for idx, frame in tqdm(enumerate(frames)):
        if idx == 0:
            height, width = frame.shape[:2]
            binned_height = height // bin_size
            binned_width = width // bin_size
            binned_frames = np.zeros((len(frames), binned_height, binned_width), dtype=np.uint16)

        reshaped_frame = frame[:binned_height * bin_size, :binned_width * bin_size].reshape(binned_height, bin_size, binned_width, bin_size)

        binned_frame = np.sum(reshaped_frame, axis=(1, 3), dtype=np.float32)
        binned_frame = binned_frame / (bin_size**2)
        binned_frames[idx,:,:] = binned_frame

    return binned_frames


def regress_drift(sig:list, time:list, save_path, wl:int=530)-> list:
    """Prepares raw data to calculate HbO and HbR: removes 
        drift if any, and normalizes around 1

    Args:
        sig (list): 1D array containing signal
        time (list): 1D array containing time
        wl (int): wavelength of light corresponding to data, necessary for saving data as npy. Defaults to 530
        filter (bool): activate Defaults to False
        
    Returns:
        list: returns only the signal in a 1D array. Time is the same.
    """
    def droite(x, a, b):
        return a*x + b
    
    print("Global regression")
    popt, pcov = curve_fit(droite, time, sig)
    pcov = None
    sig_r = sig/droite(time, *popt)

    return sig_r


def prepToCompute(frames:list, correct_motion=False, bin_size=None, regress=False):
    """_summary_

    Args:
        frames (list): _description_
        correct_motion (bool, optional): _description_. Defaults to False.
        filter (bool, optional): _description_. Defaults to False.
        bin_size (_type_, optional): _description_. Defaults to None.
        regress (bool, optional): _description_. Defaults to False.

    Returns:
        _type_: _description_
    """
    if correct_motion:
        print("Correcting motion")
        frames = motion_correction(frames[0,:,:], frames)
    if bin_size is not None:
        print("Bining pixels")
        frames = bin_pixels(frames, bin_size=bin_size)
    if regress:
        print("Normalizing")
        frames = frames/np.mean(frames, axis=0)
    
    return frames


## IOI hemodynamic analysis funtions

### Generate IOI matrices

In [ ]:
def ioi_path_length_factor(lambda1, lambda2, npoints):
    """
    Return the pathlength values in cm from Ma. et al., Phil. Trans. R. Soc. B 371: 20150360.
    Values are stored in the 'Ma_values.txt' file.
    Parameters
    ----------
    lambda1/lambda2: scalar
        Wavelengths between which to return pathlength values
    npoints: int
        Number of sampling points between lambda1/2.
    Returns
    -------
    pathlengths: 1darray
        Pathlength values in mm between lambda1 and lambda2
    """
    with open(r"C:\Users\gabri\Documents\Université\Maitrise\Projet\Widefield-Imaging-Acquisition\AnalysisPipeline\reference\Ma_values.txt", 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ')
        ma_values = []
        for row in reader:
            ma_values.append(list(map(float, row)))
    z = np.array(ma_values) #col1: wavelengths, col2: z in mm
    z[:,1] = z[:,1]/10 #Convert to cm
    if z[0,0] > lambda1:
        z = np.concatenate((np.array([[lambda1, 0], [z[0, 0]*0.9999, 0]]), z), axis=0)
    if z[-1,0] < lambda2:
        z = np.concatenate((z, np.array([[z[-1, 0]*1.00001, 0], [lambda2, 0]])), axis=0)
    xi = np.linspace(lambda1, lambda2, npoints)
    x = z[:, 0]
    pathlengths = z[:, 1]
    pathlengths = np.interp(xi, x, pathlengths)
    return pathlengths


def ioi_get_extinctions(lambda1, lambda2, npoints):
    """
    Returns the extinction coefficients (epsilon) for Hbo and HbR as a function of wavelength between lambda1 and lambda2
    Values in 1/(cm*M) by Scott Prahl at http://omlc.ogi.edu/spectra/hemoglobin/index.html are stored in the Prahl_values.txt file.
    Parameters
    ----------
    lambda1/lambda2: scalar
        Wavelengths between which to return extinction values
    npoints: int
        Number of sampling points between lambda1/2.
    Returns
    -------
    ext_HbO/HbR: 1darrays
        Extinction values for HbO and HbR in 1/(cm*M) between lambda1 and lambda2
    """
    with open(r"C:\Users\gabri\Documents\Université\Maitrise\Projet\Widefield-Imaging-Acquisition\AnalysisPipeline\reference\Prahl_values.txt", 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ')
        prahl_values = []
        for row in reader:
            prahl_values.append(list(map(float, row)))
    E = np.array(prahl_values)
    E[:,1:3] = E[:,1:3]*2.303 #correction for neperian form of the B-L law
    xi = np.linspace(lambda1, lambda2, npoints)
    x = E[:,0]
    y_HbO = E[:,1]
    y_HbR = E[:,2]
    ext_HbO = np.interp(xi, x, y_HbO)
    ext_HbR = np.interp(xi, x, y_HbR)
    return ext_HbO, ext_HbR


def ioi_epsilon_pathlength(lambda1, lambda2, npoints, baseline_hbt, baseline_hbo, baseline_hbr, filter):
    """
    Returns the extinction coefficient*pathlength curve for Hbo and HbR as a function of wavelength between lambda1 and lambda2
    Parameters
    ---------
    lambda1/2: scalars
        Wavelengths between which the system specs are defined.
    npoints: int
        Number of wavelength sampling points in system specs
    baseline_hbt/o/r: scalars
        Baseline concentrations of HbT, HbO and HbR in the brain, in uM
    filter: boolean
        Specify if the fluoresence emission filter was in place.
    Returns
    -------
    eps_pathlength: 2darray
        2d matrix of the epsilon*pathlength values for both imaging wavelengths (rows) and chromophores (columns) in 1/M.
        This matrix is used to solve the modified Beer-Lambert equation for HbO and HbR concentration changes.
    """
    os.chdir(r"C:\Users\gabri\Documents\Université\Maitrise\Projet\Widefield-Imaging-Acquisition")
    wl = np.linspace(lambda1, lambda2, npoints)
    # c_camera
    QE_moment = np.loadtxt(r"AnalysisPipeline\specs sys optique\QE_moment_5px.csv", delimiter=';')
    p = np.poly1d(np.polyfit(QE_moment[:,0], QE_moment[:,1], 10))
    c_camera = p(wl)/np.max(p(wl))
    QE_moment, p = None, None
    # c_led
    FBH530 = np.loadtxt(r"AnalysisPipeline\specs sys optique\FBH530-10.csv", skiprows=1, usecols=(0, 2), delimiter=';')
    f = interp1d(FBH530[:,0], FBH530[:,1])
    c_FBH530 = f(wl)/np.max(f(wl))
    FBH630 = np.loadtxt(r"AnalysisPipeline\specs sys optique\FBH630-10.csv", skiprows=1, usecols=(0, 2), delimiter=';')
    f = interp1d(FBH630[:,0], FBH630[:,1])
    c_FBH630 = f(wl)/np.max(f(wl))
    c_led = np.array([c_FBH530, c_FBH630])
    FBH530, FBH630, c_FBH530, c_FBH630, f = None, None, None, None, None 
    c_tot = baseline_hbt*10**-6  # Rough baseline concentrations in M
    c_pathlength = ioi_path_length_factor(lambda1, lambda2, npoints)
    c_ext_hbo, c_ext_hbr = ioi_get_extinctions(lambda1, lambda2, npoints)
    # Create vectors of values for the fits
    CHbO = baseline_hbo/baseline_hbt*c_tot*np.linspace(0, 1.5, 16) #in M
    CHbR = baseline_hbr/baseline_hbt*c_tot*np.linspace(0, 1.5, 16)
    # In this computation we neglect the fact that pathlength changes with total concentration
    # (it is fixed for a Ctot of 100e-6)
    eps_pathlength = np.zeros((2, 2))
    IHbO = np.zeros(np.shape(CHbO))
    IHbR = np.zeros(np.shape(CHbR))
    for iled in range(2):
        for iconc in range(len(CHbO)):
            IHbO[iconc] = np.sum(c_camera*c_led[iled]*np.exp(-c_ext_hbo*c_pathlength*CHbO[iconc]))
            IHbR[iconc] = np.sum(c_camera*c_led[iled]*np.exp(-c_ext_hbr*c_pathlength*CHbR[iconc]))
        IHbO = IHbO/np.max(IHbO)
        IHbR = IHbR/np.max(IHbR)
        # Compute effective eps
        # plt.plot(c_camera*c_led[iled]*np.exp(-c_ext_hbr*c_pathlength*CHbO[iconc]), 'r.')
        # plt.plot(c_camera*c_led[iled]*np.exp(-c_ext_hbo*c_pathlength*CHbR[iconc]), 'g.')
        p1 = np.polyfit(CHbO, -np.log(IHbO), 1)
        p2 = np.polyfit(CHbR, -np.log(IHbR), 1)
        HbOL = p1[0]
        HbRL = p2[0]
        eps_pathlength[iled, 0] = HbOL
        eps_pathlength[iled, 1] = HbRL
    # print(eps_pathlength)
    return eps_pathlength

### IOI analysis

In [ ]:
def convertToHb(data_green, data_red):
    """_summary_

    Args:
        data_green (_type_): _description_
        data_red (_type_): _description_

    Returns:
        _type_: _description_
    """
    lambda1 = 450 #nm
    lamba2 = 700 #nm
    npoints = 1000
    baseline_hbt = 100 #uM
    baseline_hbo = 60 #uM
    baseline_hbr = 40 #uM
    rescaling_factor = 1e6
    
    eps_pathlength = ioi_epsilon_pathlength(lambda1, lamba2, npoints, baseline_hbt, baseline_hbo, baseline_hbr, filter=None)
    Ainv = (np.linalg.pinv(eps_pathlength)*rescaling_factor).astype(np.float32)
    ln_green = -np.log(data_green.flatten())
    ln_red = -np.log(data_red.flatten())
    ln_R = np.concatenate((ln_green.reshape(1,len(ln_green)),ln_red.reshape(1,len(ln_green))))
    Hbs = np.matmul(Ainv, ln_R).astype(np.float32)
    d_HbO = Hbs[0].reshape(np.shape(data_green))
    d_HbR = Hbs[1].reshape(np.shape(data_green))
    # Protection against aberrant data points
    np.nan_to_num(d_HbO, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
    np.nan_to_num(d_HbR, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
    
    return d_HbO, d_HbR


def dHb_pipeline(data_path, save_path, correct_motion=True, bin_size=3, filter=False, regress=True):
    """_summary_

    Args:
        data_path (_type_): _description_
        save_path (_type_): _description_
        correct_motion (bool, optional): _description_. Defaults to True.
        bin_size (int, optional): _description_. Defaults to 3.
        filter (bool, optional): _description_. Defaults to True.
        cutoff (float, optional): _description_. Defaults to 0.2.
        regress (bool, optional): _description_. Defaults to True.
    """
    # process green
    print("Loading green data")
    green = create_npy_stack(data_path + "\\530", data_path, 530, saving=False)
    # green = np.load(data_path + "\\530_rawStack.npy")
    print("Green data loaded")
    green = prepToCompute(green, correct_motion=correct_motion, bin_size=bin_size, regress=regress)
    green = np.save(data_path + "\\530preped.npy", green)
    green = None
    print("Green data preped and saved")

    # process red
    print("Loading red data")
    red = create_npy_stack(data_path + "\\625", data_path, 625, saving=False)
    # red = np.load(data_path + "\\625_rawStack.npy")
    print("Red data loaded")
    red = prepToCompute(red, correct_motion=correct_motion, bin_size=bin_size, regress=regress)
    red = np.save(data_path + "\\625preped.npy", red)
    red = None
    print("Red data preped and saved")

    # convert to hb
    print("Converting to dHb")
    green = np.load(data_path + "\\530preped.npy")
    red = np.load(data_path + "\\625preped.npy")
    d_HbO, d_HbR = convertToHb(green, red)
    d_HbT = d_HbO+d_HbR
    # resample pixel values
    d_HbO = resample_pixel_value(d_HbO, 16).astype(np.uint16)
    d_HbR = resample_pixel_value(d_HbR, 16).astype(np.uint16)
    d_HbT = resample_pixel_value(d_HbT, 16).astype(np.uint16)
    Hb = np.array((d_HbO, d_HbR, d_HbT))
    # filter if needed
    if filter:
        print("Filtering")
        Hb = gaussian_filter(Hb, sigma=1, axes=(1))
    # save processed data as npy
    np.save(save_path + "\\computedHb.npy", Hb)
    # save as tiff
    print("Saving processed Hb")
    data_types = ['HbO', 'HbR', 'HbT']
    for frames, typpe in zip(Hb, data_types):
        try:
            os.mkdir(save_path + "\\"  + typpe)
        except FileExistsError:
            print("Folder already created")
        save_as_tiff(frames, typpe, save_path + "\\" + typpe)

    print("Done")


def dHb_pipeline_by_trial(data_path, save_path, event_timestamps, correct_motion=True, bin_size=3, filter=True, regress=True):

    # open data
    print("Sorting file names")
    files = [identify_files(data_path + "\\530", ".tif"), identify_files(data_path + "\\625", ".tif")]   #list:[[green files],[red files]]
    ts = [np.load(data_path + "\\530ts.npy"), np.load(data_path + "\\625ts.npy")]                        #list:[[green timestamps],[red timestamps]]
    sorted_idx_list = [create_list_trials(ts[0], event_timestamps), create_list_trials(ts[1], event_timestamps)]  #list:[green[all trials], red[all trials]]
    # print(sorted_idx_list)
    for trial_idx in range(len(sorted_idx_list[0])):
        # process green
        print("Procesing green")
        idx_inf = sorted_idx_list[0][trial_idx][0]
        idx_sup = sorted_idx_list[0][trial_idx][1]
        trial_files = files[0][idx_inf: idx_sup]
        green = create_npy_stack(data_path + "\\530", data_path, 530, saving=False, cutAroundEvent=trial_files)
        green = prepToCompute(green, correct_motion=correct_motion, bin_size=bin_size, regress=regress)
        
        # process red
        print("Processing red")
        idx_inf = sorted_idx_list[1][trial_idx][0]
        idx_sup = sorted_idx_list[1][trial_idx][1]
        trial_files = files[1][idx_inf: idx_sup]
        red = create_npy_stack(data_path + "\\625", data_path, 625, saving=False, cutAroundEvent=trial_files)
        red = prepToCompute(red, correct_motion=correct_motion, bin_size=bin_size, regress=regress) 

        # convert to hb
        print("Converting to dHb")
        d_HbO, d_HbR = convertToHb(green, red)
        d_HbT = d_HbO+d_HbR
        # resample pixel values
        d_HbO = resample_pixel_value(d_HbO, 16).astype(np.uint16)
        d_HbR = resample_pixel_value(d_HbR, 16).astype(np.uint16)
        d_HbT = resample_pixel_value(d_HbT, 16).astype(np.uint16)
        Hb = np.array((d_HbO, d_HbR, d_HbT))
        # filter if needed
        if filter:
            print("Filtering")
            Hb = gaussian_filter(Hb, sigma=1, axes=(1))
        # save processed data as npy
        np.save(save_path + "\\computedHb_trial{}.npy".format(trial_idx), Hb)
        # save as tiff
        print("Saving processed Hb")
        data_types = ['HbO', 'HbR', 'HbT']
        for frames, typpe in zip(Hb, data_types):
            try:
                os.mkdir(save_path + "\\"  + typpe)
            except FileExistsError:
                print("Folder already created")
            save_as_tiff(frames, typpe + "_trial{}_".format(trial_idx+1), save_path + "\\" + typpe)



AP_times = np.array([  12.01,   35.2 ,   46.51,   74.12,   91.14,  103.63,  114.48,
    132.14,  142.77,  169.61,  182.33,  197.83,  209.56,  223.5 ,
    239.35,  252.31,  263.77,  279.97,  297.53,  310.62,  323.38,
    335.92,  365.67,  383.93,  402.83,  417.51,  430.48,  440.9 ,
    456.7 ,  468.25,  480.64])
data_path = r"Z:\gGermain\2024-09-17\3"
dHb_pipeline_by_trial(data_path, data_path, AP_times)


## GCaMP analysis functions

In [ ]:
def compute_deltaFovF(data, data0=None):
    if data0 is not None:
        data0 = np.mean(data, axis=0)

    return (data-data0)/data0

## LSCI analysis functions

In [ ]:
def compute_speckle_contrast(raw_speckle_data, window_size=7):
    num_frames, height, width = raw_speckle_data.shape
    contrast_data = np.zeros((num_frames, height, width), dtype=np.float32)

    for frame_idx in tqdm(range(num_frames)):
        frame_data = raw_speckle_data[frame_idx]

        # Calculate local mean and standard deviation for the current frame
        local_mean = uniform_filter(frame_data, size=window_size)
        local_variance = uniform_filter(frame_data**2, size=window_size) - local_mean**2
        local_std = np.sqrt(local_variance)
        
        # Calculate speckle contrast for the current frame
        contrast_data[frame_idx] = 1/(local_std / local_mean)**2

    return contrast_data



def LSCI_pipeline(data_path, save_path):
    print("Loading data")
    data = create_npy_stack(data_path + "\\785", data_path, wl=785, saving=False)
    data = data.astype(np.float32)
    print("Computing LSCI)")
    data = compute_speckle_contrast(data, window_size=7)
    data = resample_pixel_value(data, 16).astype(np.uint16)
    print("Saving LSCI data")
    try:
        os.mkdir(save_path + "\\LSCI")
    except FileExistsError:
        print("Folder already created")
    save_as_tiff(data, save_path + "\\LSCI")